In [6]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import Model 
from tensorflow.keras.applications.densenet import DenseNet121





In [8]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: ignored

In [ ]:
train_dir = ('/content/drive/MyDrive/dataset20s/train')
val_dir = ('/content/drive/MyDrive/dataset20s/val')
test_dir = ('/content/drive/MyDrive/dataset20s/test')

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2

) 

valid_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range = 0.2,
        height_shift_range = 0.2,
       )


train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size =(IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb'
)

valid_gen = valid_datagen.flow_from_directory(
    val_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    color_mode= "rgb",
)
    


Found 567 images belonging to 3 classes.
Found 283 images belonging to 3 classes.


In [ ]:
base_model = DenseNet121(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')


In [ ]:
for layer in base_model.layers[:423]:
    layer.trainable = False

for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

0 input_2 False
1 zero_padding2d_2 False
2 conv1/conv False
3 conv1/bn False
4 conv1/relu False
5 zero_padding2d_3 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv False
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv False
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv False
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv False
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv False
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv False
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv False
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block5_

In [ ]:
from tensorflow.keras import layers 
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])


In [ ]:
from matplotlib import pyplot as plt

history = model.fit(train_gen,epochs = 50,validation_data = valid_gen)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Epoch 1/50
31/36 [========================>.....] - ETA: 1s - loss: 33.0330 - acc: 0.3901

In [ ]:
 test_datagen = ImageDataGenerator(
        rescale=1./255)


test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE,
    class_mode='categorical'
)


pred = model.predict_generator(test_gen, steps=10, verbose=1)

Found 115 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


10/10 [==============================] - 31s 3s/step


In [ ]:
final_loss, final_acc = model.evaluate(valid_gen, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

Final loss: 1.109834, final accuracy: 0.642202


In [ ]:
predicted_class = np.argmax(pred,axis=1)


In [ ]:
l = dict((v,k) for k,v in train_gen.class_indices.items())
prednames = [l[k] for k in predicted_class]


In [ ]:
filenames = test_gen.filenames


In [ ]:
import pandas as pd
finaldf = pd.DataFrame({'Filename': filenames,'Prediction': prednames})


In [ ]:
finaldf.to_csv("pred25s.csv")